# Variational autoencoder
This notebook will investiga

In [1]:
%matplotlib
import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

import pandas as pd
import seaborn as sns
from collections import defaultdict

from models.variational_autoencoder import VariationalAutoencoder
from models.autoencoder import Autoencoder
from models.continuous_bernoulli_loss import continuous_bernoulli_loss

# For dimension reduction
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

# for visualization and evaluation
from visual.latent_plane_mosaic import LatentPlaneMosaic
from visual.latent_interpolation_mosaic import LatentInterpolationMosaic
from visual.sample_scatter_gui import SampleScatterGUI

Using matplotlib backend: Qt5Agg


In [2]:
names = ['vae_10', 'vae_18', 'vae_32', 'vae_56',
         'vae_100', 'vae_180', 'vae_320', 'vae_560',
         'vae_1000']
paths = ["models\\saved\\"+name for name in names]
types = ['vae' for name in names]

model_df = pd.DataFrame(data = {
    'path' : paths,
    'ae_type' : types,
    },
    index = names,
)


In [3]:
models = []
for name, model in model_df.iterrows():
    if model['ae_type'] == "ae":
        models.append(keras.models.load_model(model['path'], 
                         custom_objects={"Autoencoder": Autoencoder,
                                         "continuous_bernoulli_loss": continuous_bernoulli_loss}))
    elif model['ae_type'] == "vae":
        models.append(keras.models.load_model(model['path'], 
                                 custom_objects={"VariationalAutoencoder": VariationalAutoencoder,
                                                 "continuous_bernoulli_loss":  continuous_bernoulli_loss}))
model_df['model'] = models    

In [4]:
num_models = model_df.shape[0]
latent_dim = 10

In [5]:
model_df.head()

,path,ae_type,model
vae_10,models\saved\vae_10,vae,<models.variational_autoencoder.VariationalAut...
vae_18,models\saved\vae_18,vae,<models.variational_autoencoder.VariationalAut...
vae_32,models\saved\vae_32,vae,<models.variational_autoencoder.VariationalAut...
vae_56,models\saved\vae_56,vae,<models.variational_autoencoder.VariationalAut...
vae_100,models\saved\vae_100,vae,<models.variational_autoencoder.VariationalAut...


## Preprocess data


In [6]:
(_, _), (test_digits, test_labels) = keras.datasets.mnist.load_data()
test_digits = np.expand_dims(test_digits, -1).astype("float32") / 255
input_shape = test_digits.shape[1:]
num_test = test_labels.shape[0]

## Load Variational autoencoder

### Print setup details

## Reconstruct and dimension reductions

In [7]:
# Fill
digit_df_list = []
dimension_df_list = []
#latent_df_list = []


for name, model in model_df.iterrows():
    print(name)
    if model['ae_type'] == "ae":
        code = model['model'].encoder(test_digits).numpy()
        code_std = np.empty(shape = code.shape)*np.nan
        
    elif model['ae_type'] == "vae":
        digit_distribution = model['model'].encoder(test_digits).numpy()
        code = digit_distribution[:,0,:]
        code_std = np.sqrt(np.exp(digit_distribution[:,1,:]))
    
    # Caclualte digit_df data
    digit_distance = np.linalg.norm(code, axis = 1)
    digit_radius = np.power(np.prod(code_std, axis = 1),1/latent_dim)
    digit_loss = tf.reduce_mean(tf.reduce_mean(tf.keras.losses.binary_crossentropy(test_digits, model['model'](test_digits)),axis = -1),axis = -1).numpy()
    digit_tsne = TSNE(n_components = 2).fit_transform(code)
    
    _digit_df = pd.DataFrame(data = {
        'model_name' : name,
        'label' : test_labels,
        'loss' : digit_loss,
        'distance' : digit_distance,
        'radius' : digit_radius,
        'tsne_0' : digit_tsne[:,0],
        'tsne_1' : digit_tsne[:,1],
    })
    digit_df_list.append(_digit_df)   
    
    
    # Calculate dimension_df data
    value_spread = np.nanstd(code, axis = 0)
    spread_mean = np.nanmean(code_std, axis = 0)
    dim_index = np.arange(latent_dim)
    
    _dimension_df = pd.DataFrame(data = {
        'model_name' : name,
        'dim_index' : dim_index,
        'value_spread' : value_spread,
        'spread_mean' : spread_mean,
    })
    
    dimension_df_list.append(_dimension_df)
    """
    # Calculate latent_df data
    latent_labels = test_labels.repeat(repeats = latent_dim)
    latent_digit_index = np.arange(num_test).repeat(repeats = latent_dim)
    latent_dim_index = np.tile(np.arange(latent_dim), num_test)
    latent_value = code.reshape(-1)
    latent_spread = code_std.reshape(-1)
    
    _latent_df = pd.DataFrame(data = {
        'model_name' : name,
        'label' : latent_labels,
        'digit_index' : latent_digit_index,
        'dim_index' : latent_dim_index,
        'value' : latent_value,
        'spread' : latent_spread
    })
    
    latent_df_list.append(_latent_df)
    """
    
        
digit_df = pd.concat(digit_df_list)
dimension_df = pd.concat(dimension_df_list)
#latent_df = pd.concat(latent_df_list)

vae_10
vae_18
vae_32
vae_56
vae_100
vae_180
vae_320
vae_560
vae_1000


<ipython-input-15-f7eddffb12ed>:15: RuntimeWarning: Mean of empty slice
  spread_mean = np.nanmean(code_std, axis = 0)


## Investigation

In [28]:
model_df.head()

,path,ae_type,model
vae_10,models\saved\vae_10,vae,<models.variational_autoencoder.VariationalAut...
vae_18,models\saved\vae_18,vae,<models.variational_autoencoder.VariationalAut...
vae_32,models\saved\vae_32,vae,<models.variational_autoencoder.VariationalAut...
vae_56,models\saved\vae_56,vae,<models.variational_autoencoder.VariationalAut...
vae_100,models\saved\vae_100,vae,<models.variational_autoencoder.VariationalAut...


### Show reconstructions

In [29]:
digit_df.head()

,model_name,label,loss,distance,radius,tsne_0,tsne_1
0,vae_10,7,0.105337,2.501040,0.680406,-6.738709,65.726761
1,vae_10,2,0.173284,2.780914,0.557840,-33.346474,15.623264
2,vae_10,1,0.055353,2.277884,0.769704,-86.091446,4.201549
3,vae_10,0,0.136475,2.014777,0.515419,29.112360,-45.153004
4,vae_10,4,0.133185,2.300354,0.639234,62.646503,18.785904


In [30]:
dimension_df.head()

,model_name,dim_index,value_spread,spread_mean
0,vae_10,0,0.753875,0.627926
1,vae_10,1,0.844394,0.526745
2,vae_10,2,0.476550,0.885191
3,vae_10,3,0.062165,0.991183
4,vae_10,4,0.080244,0.992814


In [12]:
latent_df.head()

,model_name,label,digit_index,dim_index,value,spread
0,ae,7,0,0,8.920861,NaN
1,ae,7,0,1,2.935239,NaN
2,ae,7,0,2,0.101040,NaN
3,ae,7,0,3,3.835251,NaN
4,ae,7,0,4,5.167331,NaN


In [31]:
num_col = 4
num_row = 3
num_img = num_row*num_col
rec_index = rnd.randint(num_test,size =(num_row*num_col,) )
digits = test_digits
digits = digits[rec_index]

In [32]:

model = model_df.loc['ae','model']

reconstructions = model(digits)
i = 0
for col in range(num_col):
    for row in range(num_row): 
        pair = np.concatenate((digits[i], reconstructions[i]), axis = 1)
        if row == 0: 
            ver_img = pair
        else:
            ver_img = np.concatenate((ver_img,
                                 pair), axis = 0)
        i = i + 1
    if col == 0:
        img = ver_img
    else: 
        img = np.concatenate((img,
                              ver_img), axis = 1)

sns.heatmap(img[:,:,0], vmin=0, vmax = 1)

<AxesSubplot:>

In [36]:

num_dig = 14
rec_index = rnd.randint(num_test,size =(num_dig,) )
digits = test_digits
digits = digits[rec_index]

reconstructions = []
for name, model in model_df.iterrows():
    reconstructions.append(model['model'](digits))

for dig_i in range(num_dig):
    rec_im = digits[dig_i]
    model_i = 0
    for name, model in model_df.iterrows():
        
        rec_im = np.concatenate((rec_im,
                         reconstructions[model_i][dig_i]),
                         axis = 0)
        model_i = model_i + 1
    if dig_i == 0:
        im = rec_im
    else:
        im = np.concatenate((im, rec_im), axis = 1)
    
sns.heatmap(im[:,:,0], vmin=0, vmax = 1)

<AxesSubplot:>

### Latent

In [34]:

indeces = rnd.randint(num_test,size = (3,))
indeces = [1,2,3]
ul = test_digits[indeces[0]]
ur = test_digits[indeces[1]]
dl = test_digits[indeces[2]]
z =  np.zeros(dl.shape)
corner_image = np.concatenate( (np.concatenate((ul,ur),axis = 1),
                                np.concatenate((dl,z ),axis = 1)), axis = 0)


In [35]:

model = model_df.loc['vae_10','model']
mosaic = LatentInterpolationMosaic(
                          model.encode,
                          model.decoder,
                          test_digits,
                          indeces,
                          num_row = 15,
                          num_col = 15).mosaic



sns.heatmap(mosaic[:,:,0], vmin = 0, vmax = 1)

<AxesSubplot:>

### Scatter

In [13]:
model_name = 'vae_1000'
scatter = digit_df[digit_df['model_name']==model_name][['tsne_0', 'tsne_1']].to_numpy()
SampleScatterGUI(scatter, test_labels, test_digits)


In [14]:
model_name = "vae_1000"
sns.jointplot(data = digit_df[digit_df['model_name'] == model_name ], x = 'tsne_0', y = 'tsne_1',
              hue = 'label',
              palette = 'colorblind')

In [15]:
model_name = "vae_1000"
sns.relplot(data = digit_df, x = 'tsne_0', y = 'tsne_1',
              hue = 'label',
              palette = 'colorblind',
              kind = 'scatter',
              col = 'model_name',
              col_wrap = 4,
           )

### Spread

Here we have that the first 6 principal axises are almost one and the remaining 4 are almost zero (e-3). This is combination with that the cross covariance seem to be super much zero (e-15), indicates that we have a six dimesnioal sphere in this ten dimesional latent space. Interesting to see is also if you create a latent vector in the pca-vector space. Then comparing the latent value in each dimension times the spread in each dimension, gives you the importance of that dimension in that latent point. A low importance will not have an effect on the output (kind of like a low derivative...), while a hight value will give a big difference.

In [111]:
axis_index = [0,-1]
scaling_factors = np.array([2,2])

In [112]:
model_name = 'ae'
model = model_df.loc[model_name, 'model']
latent_vectors = model_df.loc[model_name, 'latent_base'][axis_index]
latent_origin  = model_df.loc[model_name, 'latent_mean']
mosaic = LatentPlaneMosaic(model.decoder,
                  latent_vectors = (scaling_factors*latent_vectors.T).T,
                  latent_origin = latent_origin,
                  num_row = 20,
                  num_col = 20).mosaic

#fig,ax = plt.subplots(1,1)
#ax.imshow(mosaic)
sns.heatmap(mosaic[:,:,0], vmin = 0, vmax = 1)

KeyError: 'latent_base'

## Spred 

### Latent dataframe stats

In [23]:
sns.displot(data = digit_df, x = 'loss', hue = 'model_name', kind = 'kde', palette = 'colorblind')

In [10]:
sns.displot(data = digit_df, x = 'distance',hue = 'model_name', kind = 'kde')

In [11]:
sns.displot(data = digit_df, x = 'radius',hue = 'model_name', kind = 'kde')

### Summary dataframe plot

In [8]:
######### IMPORTANT ########
sns.relplot(data = dimension_df, x = 'value_spread',
            y = 'spread_mean',
            hue = 'model_name',
           size = 'model_name',
           )

### dimension dataframe plots

In [18]:
sns.relplot(data = latent_df,
           x = 'value',
           y = 'spread',
           col = 'dim_index',
           col_wrap = 4,
           hue = 'model_name')

## Generative power

In [66]:
model_name = 'ae'
mean = np.zeros(latent_dim)
mean = model_df.loc[model_name,'latent_mean']
cov  = 1*np.eye(latent_dim)
num_col = 15
num_row = 10
num_img = num_row*num_col
latent_points = rnd.multivariate_normal(mean, cov, size = (num_img))



In [67]:
model = model_df.loc[model_name, 'model']
title = "Reconstruction with autoencoder"
fig, ax = plt.subplots()
reconstructions = model.decoder(latent_points)
i = 0
for col in range(num_col):
    for row in range(num_row): 
        if row == 0: 
            ver_img = reconstructions[i]
        else:
            ver_img = np.concatenate((ver_img,
                                 reconstructions[i]), axis = 0)
        i = i + 1
    if col == 0:
        img = ver_img
    else: 
        img = np.concatenate((img,
                              ver_img), axis = 1)
sns.heatmap(img[:,:,0])



<AxesSubplot:>

In [ ]:
mean = np.zeros(latent_dim)
cov  = 1*np.eye(latent_dim)
num_dig = 5
latent_points = rnd.multivariate_normal(mean, cov, size = (num_dig))

reconstructions = []
for name, model in model_df.iterrows():
    reconstructions.append(model['model'].decoder(latent_points))
    
fig, axs = plt.subplots(num_models, num_dig, sharex = True, sharey = True)
for model_i in range(num_models):
    for dig_i in range(num_dig):
        ax = axs[model_i, dig_i]
        sns.heatmap(reconstructions[model_i][dig_i][:,:,0], ax = ax)



array([ 0.04270896, -0.02646526, -0.01551382,  0.00895875,  0.00756909,
       -0.00550758, -0.01186571,  0.00023721, -0.0082873 ,  0.0526603 ],
      dtype=float32)

In [23]:
model_df.head()

,path,ae_type,model
ae,models\saved\ae,ae,<models.autoencoder.Autoencoder object at 0x00...
vae_10,models\saved\vae_10,vae,<models.variational_autoencoder.VariationalAut...
vae_100,models\saved\vae_100,vae,<models.variational_autoencoder.VariationalAut...
vae_1000,models\saved\vae_1000,vae,<models.variational_autoencoder.VariationalAut...


In [24]:
latent_df.head()

,model_name,label,loss,distance,radius
0,ae,7,0.070490,15.820858,NaN
1,ae,2,0.117689,9.960896,NaN
2,ae,1,0.038200,24.145765,NaN
3,ae,0,0.096915,8.166095,NaN
4,ae,4,0.084998,14.815680,NaN


In [25]:
dimension_df.head()

,model_name,label,digit_index,dim_index,value,std
0,ae,7,0,0,8.920861,NaN
1,ae,7,0,1,2.935239,NaN
2,ae,7,0,2,0.101040,NaN
3,ae,7,0,3,3.835251,NaN
4,ae,7,0,4,5.167331,NaN


In [26]:
summary_df.head()

,model_name,dim_index,value_spread,spread_mean
0,ae,0,3.059391,NaN
1,ae,1,3.437786,NaN
2,ae,2,2.502900,NaN
3,ae,3,2.846762,NaN
4,ae,4,2.707471,NaN
